In [1]:
import os
# pyspark --packages graphframes:graphframes:0.3.0-spark2.0-s_2.11 --jars graphframes-0.3.0-spark2.0-s_2.11.jar
pyspark_submit_args = '--packages graphframes:graphframes:0.3.0-spark2.0-s_2.11 pyspark-shell'# --jars graphframes-0.8.0-spark3.0-s_2.12.jar'
pyspark_submit_args = '--jars graphframes-0.8.0-spark3.0-s_2.12.jar pyspark-shell'

os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args
import findspark
findspark.init()
findspark.find()
import pyspark
conf = pyspark.SparkConf()
sc = pyspark.SparkContext(conf=conf)

# from pyspark import *


In [2]:
from pyspark import *
from pyspark.sql import *
from graphframes import *
sparkSession = SparkSession.builder.appName('A4').master('local').getOrCreate()
# spark = SparkSession.builder.appName('BDA_A2').master('local').getOrCreate()

In [3]:
dataset = open('soc-Epinions1.txt', 'r')
lines = dataset.readlines()
lines = lines[4:] # skipping the comments in file

In [4]:
vertices = set()
edges = []
for line in lines:
    source, dest = line.split()
    vertices.add((source,))
    vertices.add((dest,))
    edges.append((source, dest))
#     print(source, dest)
#     break
vertices = list(vertices)

In [5]:
vertices_df = sparkSession.createDataFrame(vertices, ["id"])
edges_df = sparkSession.createDataFrame(edges, ["src", "dst"])

Py4JJavaError: An error occurred while calling o26.applySchemaToPythonRDD.
: java.lang.NoClassDefFoundError: scala/Product$class
	at scala.reflect.internal.Symbols$SymbolOps.<init>(Symbols.scala:3631)
	at scala.reflect.internal.Symbols$class.$init$(Symbols.scala:3632)
	at scala.reflect.internal.SymbolTable.<init>(SymbolTable.scala:16)
	at scala.reflect.runtime.JavaUniverse.<init>(JavaUniverse.scala:16)
	at scala.reflect.runtime.package$.universe$lzycompute(package.scala:17)
	at scala.reflect.runtime.package$.universe(package.scala:17)
	at org.apache.spark.sql.catalyst.ScalaReflection$.<init>(ScalaReflection.scala:48)
	at org.apache.spark.sql.catalyst.ScalaReflection$.<clinit>(ScalaReflection.scala)
	at org.apache.spark.sql.catalyst.encoders.RowEncoder$.serializerFor(RowEncoder.scala:77)
	at org.apache.spark.sql.catalyst.encoders.RowEncoder$.apply(RowEncoder.scala:66)
	at org.apache.spark.sql.Dataset$.ofRows(Dataset.scala:88)
	at org.apache.spark.sql.SparkSession.internalCre

In [ ]:
graph = GraphFrame(vertices_df, edges_df)

In [ ]:
!java --version